# GMW: International status data processing.  
This notebooks documents the data processing pipeline for the upcoming international status widget.  
This widget consists on a series of sentences detailing a number of agreements and targets that involved countries could have taken up.  

  
For this, the application will require the data in the following format:  

**data**  
```pledge_type``` *string*  
```base_years``` *string*  
```target_years``` *string*  
```ndc_target``` *string*  
```ndc_target_url``` *string*  
```ndc_reduction_target``` *string*  
```ndc_blurb``` *string*  
```ndc_updated``` *boolean*  
```ndc_mitigation``` *boolean*  
```ndc_adaptation``` *boolean*  
```ipcc_wetlands_suplement``` *string*  
   
**metadata**  
   ```location_id``` *number* 

 

In [50]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import requests


## 1) Data loading  

Data can be retrieved directly from the google cloud bucket of the project.

In [51]:
data_url = 'https://storage.googleapis.com/mangrove_atlas/widget_data/Climate_Policy_Tool_Data_v20220901.csv'
original_data = pd.read_csv(data_url)
#original_data = pd.read_csv('../../../../data/Climate_Policy_Tool_Data_prep.csv')

original_data.columns = original_data.columns.str.replace(' ', '_').str.replace('%', 'pct').str.lower()
original_data.columns

Index(['id', 'type', 'iso', 'name', 'total_organic_carbon',
       '2016_mangrove_extent_(ha)', 'ndc_pct_reduction_target',
       'ndc_target_(mtco2e/yr)', 'ncs_submission_language', 'pledge_type',
       'base_year', 'target_year', 'ndc_target_url', 'pledge_summary',
       'ndc_blurb', 'climate_vulnerability_rank_(index)', 'ndc:_first/updated',
       'ndc:includes/doesn't_include', 'ndc_mitigation_(y/n)',
       'ndc_adaptation_(y/n)', 'frel', 'forest_or_wetland',
       'investible_blue_carbon_extent_@_$5/ton',
       'investible_blue_carbon_extent_@_$5/ton.1',
       'investible_blue_carbon_extent_@_$10/ton',
       'investible_blue_carbon_extent_@_$10/ton.1',
       'investible_blue_carbon_extent_(diff_btw._$10/ton_and_$5/ton)',
       'extent_within_pa', 'remaining_extent', 'wetland_supplement',
       'ghg_inventories', 'mangrove_considerations_in_national_policies',
       'emissions_by_land_sector_by_type',
       'avoided_loss_(emissions_from_mangrove_loss)_(mtco2e_yr-1)',


In [52]:
selected_data = original_data[['iso', 'name',
                             '2016_mangrove_extent_(ha)', 'ndc_pct_reduction_target',
                             'ndc_target_(mtco2e/yr)', 'ncs_submission_language', 'pledge_type',
                             'base_year', 'target_year', 'ndc_target_url', 'pledge_summary',
                             'ndc_blurb', 'climate_vulnerability_rank_(index)', 'ndc:_first/updated',
                             'ndc:includes/doesn\'t_include', 'ndc_mitigation_(y/n)',
                             'ndc_adaptation_(y/n)', 'frel', 'forest_or_wetland', 'wetland_supplement',
                             'ghg_inventories', 'frel_year']].copy()



selected_data.columns = ['iso', 'country',
                      '2016_mangrove_extent', 'ndc_pct_reduction_target',
                      'ndc_target', 'ncs_submission_language', 'pledge_type',
                      'base_year', 'target_year', 'ndc_target_url', 'pledge_summary',
                      'ndc_blurb', 'climate_vulnerability_rank', 'ndc_first_updated',
                      'ndc_includes', 'ndc_mitigation_y/n',
                      'ndc_adaptation_y/n', 'frel', 'forest_or_wetland', 'wetland_supplement',
                      'ghg_inventories', 'frel_year']
selected_data.head()

,iso,country,2016_mangrove_extent,ndc_pct_reduction_target,ndc_target,ncs_submission_language,pledge_type,base_year,target_year,ndc_target_url,...,climate_vulnerability_rank,ndc_first_updated,ndc_includes,ndc_mitigation_y/n,ndc_adaptation_y/n,frel,forest_or_wetland,wetland_supplement,ghg_inventories,frel_year
0,AGO,Angola,36527.73,14,96.65,NaN,a GHG target,2005,2025,https://www4.unfccc.int/sites/NDCStaging/pages...,...,106.67,Updated,Includes,No,Yes,NaN,NaN,Has not,NaN,NaN
1,ATG,Antigua & Barbuda,889.68,25,NaN,NaN,Non-GHG target and actions,1990,2030,https://www4.unfccc.int/sites/NDCStaging/pages...,...,74.50,NaN,Doesn't include,No,No,NaN,NaN,Has not,NaN,NaN
2,AUS,Australia,967262.58,43,133.74,NaN,a GHG Target,2005,2030,https://www4.unfccc.int/sites/NDCStaging/pages...,...,52.00,Updated,Includes,Yes,No,NaN,not specified,Has,NaN,NaN
3,BHR,Bahrain,82.03,NaN,NaN,NaN,Actions only,NaN,NaN,https://www4.unfccc.int/sites/NDCStaging/pages...,...,125.00,NaN,Doesn't include,No,No,NaN,NaN,Has not,NaN,NaN
4,BGD,Bangladesh,404613.41,6.73,27.56,NaN,a GHG target,2030,2030,https://www4.unfccc.int/sites/NDCStaging/pages...,...,25.00,NaN,Doesn't include,No,No,374253,NaN,Has not,NaN,2019


In [53]:
selected_data.dtypes

iso                            object
country                        object
2016_mangrove_extent          float64
ndc_pct_reduction_target       object
ndc_target                     object
ncs_submission_language        object
pledge_type                    object
base_year                      object
target_year                    object
ndc_target_url                 object
pledge_summary                 object
ndc_blurb                      object
climate_vulnerability_rank    float64
ndc_first_updated              object
ndc_includes                   object
ndc_mitigation_y/n             object
ndc_adaptation_y/n             object
frel                           object
forest_or_wetland              object
wetland_supplement             object
ghg_inventories               float64
frel_year                      object
dtype: object

## Populate data model

In [54]:
data_model = selected_data[['iso', 'country']].copy()
data_model.head(3)

,iso,country
0,AGO,Angola
1,ATG,Antigua & Barbuda
2,AUS,Australia


### Pledges and NDC info  
This data is taken directly from the table  

```pledge_type``` *string*  
```base_years``` *string*  
```target_years``` *string*  
```ndc_target``` *string*  
```ndc_target_url``` *string*  
```ndc_reduction_target``` *string*  
```ndc_pct_reduction_target``` *string*   
```ndc_blurb``` *string*  

In [55]:
data_model['pledge_type'] = selected_data['pledge_type'].str.lower().str.replace('ghg', 'GHG')
data_model['base_years'] = selected_data['base_year'].str.lower().str.replace('&', 'and')
data_model['target_years'] = selected_data['target_year'].str.lower().str.replace('&', 'and')
data_model['ndc_target'] = selected_data['ndc_target'].str.lower()
data_model['ndc_reduction_target'] = selected_data['ndc_pct_reduction_target'].str.lower().str.replace('&', 'and').str.replace(' and', '% and')
data_model['ndc_target_url'] = selected_data['ndc_target_url'].str.lower()
data_model['pledge_summary'] = selected_data['pledge_summary']
data_model['ndc_blurb'] = selected_data['ndc_blurb']

data_model.head(3)

,iso,country,pledge_type,base_years,target_years,ndc_target,ndc_reduction_target,ndc_target_url,pledge_summary,ndc_blurb
0,AGO,Angola,a GHG target,2005,2025,96.65,14,https://www4.unfccc.int/sites/ndcstaging/pages...,"""Angola plans to reduce GHG emissions up to 35...","""Angola plans to reduce GHG emissions up to 35..."
1,ATG,Antigua & Barbuda,non-GHG target and actions,1990,2030,NaN,25,https://www4.unfccc.int/sites/ndcstaging/pages...,Antigua and Barbuda communicated that it would...,"Conditional Adaptation Targets "" (1) By 2025, ..."
2,AUS,Australia,a GHG target,2005,2030,133.74,43,https://www4.unfccc.int/sites/ndcstaging/pages...,Australia communicated a target of 5 per cent ...,"""Under a Paris Agreement applicable to all, Au..."


#### First sentence
"`Country` NDC pledge contains `pledge_type`"  
If `pledge_type` is empty, ignore this sencente

In [56]:
def sentence_1(df, country):
    df = df[df['country'] == country]
    if (df['pledge_type'].notnull()).any():
        return f'{country} NDC pledge contains {df[df["country"] == country]["pledge_type"].values[0]}'
    else:
        return f'No NDC pledge'

In [57]:
for country in data_model['country'].values[0:12]:
    print(sentence_1(data_model, country))

Angola NDC pledge contains a GHG target
Antigua & Barbuda NDC pledge contains non-GHG target and actions
Australia NDC pledge contains a GHG target
Bahrain NDC pledge contains actions only
Bangladesh NDC pledge contains a GHG target
Belize NDC pledge contains a non-GHG target and actions
Benin NDC pledge contains actions
No NDC pledge
Brazil NDC pledge contains a GHG target
No NDC pledge
Cambodia NDC pledge contains a GHG target and actions
No NDC pledge


### Second sentence  
Multiple posibilities based on the info from:  
```ndc_target``` *string*  
```ndc_reduction_target``` *string*  
```base_years``` *string*  
```target_years``` *string* 
    
      
- "The GHG target is a `ndc_reduction_target`% reduction from a baseline in `base_years` by target year `target_years`. This represents a reduction of `ndc_target` mtCO2e/yr."  
  
- "The GHG target is a `ndc_reduction_target`% reduction from a baseline in `base_years` by target year `target_years`."
  
- "The GHG target represents a reduction of `ndc_target` mtCO2e/yr from a baseline in `base_years` by target year `target_years`."  
(If either `base_years` or `target_years` (or both) missing, just delete the associated clause(s))

In [58]:
def sentence_2(df, country):
    df = df[df['country'] == country]
    ndc_target = df['ndc_target'].values[0]
    ndc_reduction_target = df['ndc_reduction_target'].values[0]
    base_years = df['base_years'].values[0]
    target_years = df['target_years'].values[0]


    if (all(i == i for i in [ndc_target, ndc_reduction_target, base_years, target_years])):
        return f'The GHG target is a {ndc_reduction_target}% reduction from a baseline in {base_years} by target year/s {target_years}. This represents a reduction of {ndc_target} mtCO2e/yr.'
    elif (all(i == i for i in [ndc_reduction_target, base_years, target_years])):
        return f'The GHG target is a {ndc_reduction_target}% reduction from a baseline in {base_years} by target year/s {target_years}'
    elif (all(i == i for i in [ndc_target, base_years, target_years])):
        return f'The GHG target represents a reduction of {ndc_target} mtCO2e/yr from a baseline in {base_years} by target year/s {target_years}.'
    elif (all(i == i for i in [ndc_target, base_years])):
        return f'The GHG target represents a reduction of {ndc_target} mtCO2e/yr from a baseline in {base_years}.'
    elif (all(i == i for i in [ndc_target, target_years])):
        return f'The GHG target represents a reduction of {ndc_target} mtCO2e/yr by target year/s {target_years}.'
    elif (np.isnan(ndc_target)):
        return 'No data'
    else:
        return f'The GHG target represents a reduction of {ndc_target} mtCO2e/yr'


In [59]:
for country in data_model.country.values[0:10]:
   
    print(country + ": " + sentence_2(data_model, country))
    print(' ')

Angola: The GHG target is a 14% reduction from a baseline in 2005 by target year/s 2025. This represents a reduction of 96.65 mtCO2e/yr.
 
Antigua & Barbuda: The GHG target is a 25% reduction from a baseline in 1990 by target year/s 2030
 
Australia: The GHG target is a 43% reduction from a baseline in 2005 by target year/s 2030. This represents a reduction of 133.74 mtCO2e/yr.
 
Bahrain: No data
 
Bangladesh: The GHG target is a 6.73% reduction from a baseline in 2030 by target year/s 2030. This represents a reduction of 27.56 mtCO2e/yr.
 
Belize: No data
 
Benin: The GHG target is a 21.4% reduction from a baseline in 2030 by target year/s 2030. This represents a reduction of 4.82 mtCO2e/yr.
 
Bonaire, Sint-Eustasius, Saba: No data
 
Brazil: The GHG target is a 37% and 50% reduction from a baseline in 2005 by target year/s 2025 and 2030. This represents a reduction of 220 and 480 mtCO2e/yr.
 
Brunei: No data
 


In [60]:
sentence_2(data_model, 'Costa Rica')

'The GHG target represents a reduction of 9.11 mtCO2e/yr by target year/s 2030.'

### Add NDC first/updated data and other related info  
```ndc_updated``` *boolean*  
```ndc``` *boolean*  
```ndc_mitigation``` *boolean*  
```ndc_adaptation``` *boolean*  
```ipcc_wetlands_suplement``` *string* 

ndc_first_updated	ndc_includes	ndc_mitigation_y/n	ndc_adaptation_y/n	frel	forest_or_wetland	wetland_supplement

In [61]:
selected_data['ndc_mitigation_y/n'].value_counts()

Yes    21
No     17
No      1
Name: ndc_mitigation_y/n, dtype: int64

In [62]:
data_model['ndc_blurb'] = data_model['ndc_blurb'].str.replace('"', '')

In [63]:
data_model['ndc'] = np.where(data_model['ndc_blurb'].notnull(), True, False)
data_model['ndc_updated'] = np.where(selected_data['ndc_target_url'].notnull(), np.where(selected_data['ndc_first_updated'] == 'Updated', True, False), False).astype(bool)
data_model['ndc_mitigation'] = np.where(selected_data['ndc_target_url'].notnull(), np.where(selected_data['ndc_mitigation_y/n'] == 'Yes', True, False), False).astype(bool)
data_model['ndc_adaptation'] = np.where(selected_data['ndc_target_url'].notnull(), np.where(selected_data['ndc_adaptation_y/n'] == 'Yes', True, False), False).astype(bool)
data_model['ipcc_wetlands_suplement'] = selected_data['wetland_supplement'].str.lower()


data_model[['country','ndc_blurb', 'ndc', 'ndc_updated', 'ndc_mitigation',
       'ndc_adaptation', 'ipcc_wetlands_suplement']].head(15)

,country,ndc_blurb,ndc,ndc_updated,ndc_mitigation,ndc_adaptation,ipcc_wetlands_suplement
0,Angola,Angola plans to reduce GHG emissions up to 35%...,True,True,False,True,has not
1,Antigua & Barbuda,"Conditional Adaptation Targets (1) By 2025, i...",True,False,False,False,has not
2,Australia,"Under a Paris Agreement applicable to all, Aus...",True,True,True,False,has
3,Bahrain,The Kingdom of Bahrain communicated in its NDC...,True,False,False,False,has not
4,Bangladesh,The NDC of Bangladesh consists of the followin...,True,False,False,False,has not
5,Belize,Belize mitigation potential is framed on an ac...,True,False,False,False,has not
6,Benin,The Benin plans to reduce overall cumulative g...,True,False,False,False,has not
7,"Bonaire, Sint-Eustasius, Saba",NaN,False,False,False,False,has not
8,Brazil,Brazil intends to commit to reduce greenhouse ...,True,False,False,False,has not
9,Brunei,Brunei Darussalams Intended Nationally Determi...,True,True,True,True,has not


### Third sentence
Various combinations depending on TRUE / FALSE values on `ndc`, `ndc_updated`, `ndc_adaptation` and `ndc_mitigation`:

`country` `first`/`updated` NDC pledge...  
- includes coastal and marine NBS for `both mitigation and adaptation`
- includes coastal and marine NBS for `mitigation`/`adaptation`
- `doesn't include` coastal and marine NBS

In [64]:
def sentence_3(df, country):
    df = df[df['country'] == country]

    ndc = df['ndc'].values[0]
    ndc_updated = df['ndc_updated'].values[0]
    ndc_mitigation = df['ndc_mitigation'].values[0]
    ndc_adaptation = df['ndc_adaptation'].values[0]

    if (ndc):
        if(ndc_updated):
            fst_upd = 'updated'
        else:
            fst_upd = 'first'
        if(ndc_mitigation & ndc_adaptation): 
            inc = 'includes coastal and marine NBS for both mitigation and adaptation'
        elif(ndc_mitigation):
            inc = 'includes coastal and marine NBS for mitigation'
        elif(ndc_adaptation):
            inc = 'includes coastal and marine NBS for adaptation'
        else:
            inc = 'doesn\'t include coastal and marine NBS'

        return f'{country} {fst_upd} NDC pledge {inc}.'
    else:
        return f'No NDC pledge'

    

In [65]:
for country in data_model.country.values[0:15]:
    print(sentence_3(data_model, country))
    print(' ')

Angola updated NDC pledge includes coastal and marine NBS for adaptation.
 
Antigua & Barbuda first NDC pledge doesn't include coastal and marine NBS.
 
Australia updated NDC pledge includes coastal and marine NBS for mitigation.
 
Bahrain first NDC pledge doesn't include coastal and marine NBS.
 
Bangladesh first NDC pledge doesn't include coastal and marine NBS.
 
Belize first NDC pledge doesn't include coastal and marine NBS.
 
Benin first NDC pledge doesn't include coastal and marine NBS.
 
No NDC pledge
 
Brazil first NDC pledge doesn't include coastal and marine NBS.
 
Brunei updated NDC pledge includes coastal and marine NBS for both mitigation and adaptation.
 
Cambodia updated NDC pledge includes coastal and marine NBS for both mitigation and adaptation.
 
Cameroon first NDC pledge doesn't include coastal and marine NBS.
 
No NDC pledge
 
China first NDC pledge doesn't include coastal and marine NBS.
 
Colombia updated NDC pledge includes coastal and marine NBS for both mitiga

### Fourth sentence
Based on content from `ipcc_wetlands_suplement`  
`country` `has` / `has not` implemented the IPCC wetlands supplement.

In [66]:
def sentence_4(data_model, country):
    df = data_model[data_model['country'] == country]
    return f'{country} {df["ipcc_wetlands_suplement"].values[0]} implemented the IPCC Wetlands Supplement.'

In [67]:
for country in data_model.country.values[0:5]:
    print(sentence_4(data_model, country))
    print(' ')

Angola has not implemented the IPCC Wetlands Supplement.
 
Antigua & Barbuda has not implemented the IPCC Wetlands Supplement.
 
Australia has implemented the IPCC Wetlands Supplement.
 
Bahrain has not implemented the IPCC Wetlands Supplement.
 
Bangladesh has not implemented the IPCC Wetlands Supplement.
 


## FRELs data  
frel: `frel`,  
fow: `forest_wetlands` (only when it is defined),  
year_frel: `frel_years`,


In [68]:
selected_data.columns

Index(['iso', 'country', '2016_mangrove_extent', 'ndc_pct_reduction_target',
       'ndc_target', 'ncs_submission_language', 'pledge_type', 'base_year',
       'target_year', 'ndc_target_url', 'pledge_summary', 'ndc_blurb',
       'climate_vulnerability_rank', 'ndc_first_updated', 'ndc_includes',
       'ndc_mitigation_y/n', 'ndc_adaptation_y/n', 'frel', 'forest_or_wetland',
       'wetland_supplement', 'ghg_inventories', 'frel_year'],
      dtype='object')

In [69]:
frel = selected_data[['country', 'frel', 'forest_or_wetland', 'frel_year']].copy()
frel['fow'] = np.where(frel.forest_or_wetland.str.lower() == 'forest', 'forest', np.where(frel.forest_or_wetland.str.lower() == 'wetland', 'wetland', ''))

frel['frel'] = frel['frel'].str.replace('ﾠ','')
frel['frel'] = frel['frel'].astype(float)
frel.head(10)


,country,frel,forest_or_wetland,frel_year,fow
0,Angola,NaN,NaN,NaN,
1,Antigua & Barbuda,NaN,NaN,NaN,
2,Australia,NaN,not specified,NaN,
3,Bahrain,NaN,NaN,NaN,
4,Bangladesh,3.742530e+05,NaN,2019,
5,Belize,5.583087e+06,NaN,2020,
6,Benin,NaN,NaN,NaN,
7,"Bonaire, Sint-Eustasius, Saba",NaN,NaN,NaN,
8,Brazil,7.517805e+08,Forest,2019,forest
9,Brunei,NaN,NaN,NaN,


In [70]:
frel.dtypes

country               object
frel                 float64
forest_or_wetland     object
frel_year             object
fow                   object
dtype: object

In [71]:
#Check values (no Wetlands on original data)
frel['fow'].value_counts()

          79
forest    22
Name: fow, dtype: int64

In [72]:
frel.rename(columns={'frel_year':'year_frel'}, inplace=True)
frel.drop(columns=['forest_or_wetland'], inplace=True)
frel.head(1)

,country,frel,year_frel,fow
0,Angola,NaN,NaN,


In [73]:
data_model = pd.merge(data_model, frel, on='country', how='left')

## Final format  
Review of the final data (except for country / location info preparation for API ingestion), and the generated text for example countries

### Data Model

In [74]:
data_model.head(6)

,iso,country,pledge_type,base_years,target_years,ndc_target,ndc_reduction_target,ndc_target_url,pledge_summary,ndc_blurb,ndc,ndc_updated,ndc_mitigation,ndc_adaptation,ipcc_wetlands_suplement,frel,year_frel,fow
0,AGO,Angola,a GHG target,2005,2025,96.65,14,https://www4.unfccc.int/sites/ndcstaging/pages...,"""Angola plans to reduce GHG emissions up to 35...",Angola plans to reduce GHG emissions up to 35%...,True,True,False,True,has not,NaN,NaN,
1,ATG,Antigua & Barbuda,non-GHG target and actions,1990,2030,NaN,25,https://www4.unfccc.int/sites/ndcstaging/pages...,Antigua and Barbuda communicated that it would...,"Conditional Adaptation Targets (1) By 2025, i...",True,False,False,False,has not,NaN,NaN,
2,AUS,Australia,a GHG target,2005,2030,133.74,43,https://www4.unfccc.int/sites/ndcstaging/pages...,Australia communicated a target of 5 per cent ...,"Under a Paris Agreement applicable to all, Aus...",True,True,True,False,has,NaN,NaN,
3,BHR,Bahrain,actions only,NaN,NaN,NaN,NaN,https://www4.unfccc.int/sites/ndcstaging/pages...,NaN,The Kingdom of Bahrain communicated in its NDC...,True,False,False,False,has not,NaN,NaN,
4,BGD,Bangladesh,a GHG target,2030,2030,27.56,6.73,https://www4.unfccc.int/sites/ndcstaging/pages...,"""The NDC of Bangladesh consists of the followi...",The NDC of Bangladesh consists of the followin...,True,False,False,False,has not,374253.0,2019,
5,BLZ,Belize,a non-GHG target and actions,NaN,2030,NaN,NaN,https://www4.unfccc.int/sites/ndcstaging/pages...,NaN,Belize mitigation potential is framed on an ac...,True,False,False,False,has not,5583087.0,2020,


### Full text 

In [75]:
def full_text(df, country):
    df = df[df['country'] == country]
    s1 = sentence_1(df, country)
    s2 = sentence_2(df, country)
    s3 = sentence_3(df, country)
    s4 = sentence_4(df, country)

    print(s1)
    print(s2)
    print(' ')
    print(s3)
    print(' ')
    print(s4)

In [76]:
full_text(data_model, 'Brazil')

Brazil NDC pledge contains a GHG target
The GHG target is a 37% and 50% reduction from a baseline in 2005 by target year/s 2025 and 2030. This represents a reduction of 220 and 480 mtCO2e/yr.
 
Brazil first NDC pledge doesn't include coastal and marine NBS.
 
Brazil has not implemented the IPCC Wetlands Supplement.


In [77]:
full_text(data_model, 'Indonesia')

Indonesia NDC pledge contains a GHG target and actions
The GHG target represents a reduction of 1181.21 mtCO2e/yr by target year/s 2020.
 
Indonesia first NDC pledge doesn't include coastal and marine NBS.
 
Indonesia has not implemented the IPCC Wetlands Supplement.


In [78]:
full_text(data_model, 'Australia')

Australia NDC pledge contains a GHG target
The GHG target is a 43% reduction from a baseline in 2005 by target year/s 2030. This represents a reduction of 133.74 mtCO2e/yr.
 
Australia updated NDC pledge includes coastal and marine NBS for mitigation.
 
Australia has implemented the IPCC Wetlands Supplement.


## Add Location IDs and get final format

In [79]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [80]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [81]:
#dataLocation = requests.get('https://mangrove-atlas-api.herokuapp.com/api/v2/locations').json()['data']
#dataLocation = requests.get('http://192.168.50.115:3000/api/v2/locations').json()['data']
#locations = pd.DataFrame(dataLocation)
#locations

In [82]:
data_model.head(3)

,iso,country,pledge_type,base_years,target_years,ndc_target,ndc_reduction_target,ndc_target_url,pledge_summary,ndc_blurb,ndc,ndc_updated,ndc_mitigation,ndc_adaptation,ipcc_wetlands_suplement,frel,year_frel,fow
0,AGO,Angola,a GHG target,2005,2025,96.65,14,https://www4.unfccc.int/sites/ndcstaging/pages...,"""Angola plans to reduce GHG emissions up to 35...",Angola plans to reduce GHG emissions up to 35%...,True,True,False,True,has not,NaN,NaN,
1,ATG,Antigua & Barbuda,non-GHG target and actions,1990,2030,NaN,25,https://www4.unfccc.int/sites/ndcstaging/pages...,Antigua and Barbuda communicated that it would...,"Conditional Adaptation Targets (1) By 2025, i...",True,False,False,False,has not,NaN,NaN,
2,AUS,Australia,a GHG target,2005,2030,133.74,43,https://www4.unfccc.int/sites/ndcstaging/pages...,Australia communicated a target of 5 per cent ...,"Under a Paris Agreement applicable to all, Aus...",True,True,True,False,has,NaN,NaN,


In [94]:
final_data = pd.merge(data_model,locations[['iso', 'location_idn']], on='iso', how='left')
final_data = pd.merge(final_data,api_locs, on='location_idn', how='left')
final_data.drop(columns=['iso', 'country', 'location_idn'], inplace=True)
final_data.rename(columns={'id':'location_id'}, inplace=True)
final_data.head()

,pledge_type,base_years,target_years,ndc_target,ndc_reduction_target,ndc_target_url,pledge_summary,ndc_blurb,ndc,ndc_updated,ndc_mitigation,ndc_adaptation,ipcc_wetlands_suplement,frel,year_frel,fow,location_id
0,a GHG target,2005,2025,96.65,14,https://www4.unfccc.int/sites/ndcstaging/pages...,"""Angola plans to reduce GHG emissions up to 35...",Angola plans to reduce GHG emissions up to 35%...,True,True,False,True,has not,NaN,NaN,,2029
1,non-GHG target and actions,1990,2030,NaN,25,https://www4.unfccc.int/sites/ndcstaging/pages...,Antigua and Barbuda communicated that it would...,"Conditional Adaptation Targets (1) By 2025, i...",True,False,False,False,has not,NaN,NaN,,3095
2,a GHG target,2005,2030,133.74,43,https://www4.unfccc.int/sites/ndcstaging/pages...,Australia communicated a target of 5 per cent ...,"Under a Paris Agreement applicable to all, Aus...",True,True,True,False,has,NaN,NaN,,2441
3,actions only,NaN,NaN,NaN,NaN,https://www4.unfccc.int/sites/ndcstaging/pages...,NaN,The Kingdom of Bahrain communicated in its NDC...,True,False,False,False,has not,NaN,NaN,,4559
4,a GHG target,2030,2030,27.56,6.73,https://www4.unfccc.int/sites/ndcstaging/pages...,"""The NDC of Bangladesh consists of the followi...",The NDC of Bangladesh consists of the followin...,True,False,False,False,has not,374253.0,2019,,4587


In [96]:
final_data.dtypes

pledge_type                 object
base_years                  object
target_years                object
ndc_target                  object
ndc_reduction_target        object
ndc_target_url              object
pledge_summary              object
ndc_blurb                   object
ndc                           bool
ndc_updated                   bool
ndc_mitigation                bool
ndc_adaptation                bool
ipcc_wetlands_suplement     object
frel                       float64
year_frel                   object
fow                         object
location_id                  int64
dtype: object

In [97]:
final_data['ndc_target'] = final_data['ndc_target'].str.replace('na','')

In [98]:
final_data_long = pd.melt(final_data,id_vars='location_id').rename(columns={'variable':'indicator'})
final_data_long

,location_id,indicator,value
0,2029,pledge_type,a GHG target
1,3095,pledge_type,non-GHG target and actions
2,2441,pledge_type,a GHG target
3,4559,pledge_type,actions only
4,4587,pledge_type,a GHG target
...,...,...,...
1611,3248,fow,
1612,3582,fow,
1613,1681,fow,forest
1614,3037,fow,


In [99]:
final_data_long[final_data_long['location_id'] == 3767]

,location_id,indicator,value
53,3767,pledge_type,NaN
154,3767,base_years,NaN
255,3767,target_years,NaN
356,3767,ndc_target,NaN
457,3767,ndc_reduction_target,NaN
558,3767,ndc_target_url,https://www4.unfccc.int/sites/ndcstaging/pages...
659,3767,pledge_summary,NaN
760,3767,ndc_blurb,Mozambique communicated its mitigation contrib...
861,3767,ndc,True
962,3767,ndc_updated,False


In [100]:
final_data_long = final_data_long[final_data_long.value != '']
final_data_long = final_data_long[final_data_long.value.notnull()]
final_data_long

,location_id,indicator,value
0,2029,pledge_type,a GHG target
1,3095,pledge_type,non-GHG target and actions
2,2441,pledge_type,a GHG target
3,4559,pledge_type,actions only
4,4587,pledge_type,a GHG target
...,...,...,...
1578,3021,fow,forest
1594,2558,fow,forest
1598,2292,fow,forest
1600,4511,fow,forest


In [101]:
final_data_long.indicator.unique()

array(['pledge_type', 'base_years', 'target_years', 'ndc_target',
       'ndc_reduction_target', 'ndc_target_url', 'pledge_summary',
       'ndc_blurb', 'ndc', 'ndc_updated', 'ndc_mitigation',
       'ndc_adaptation', 'ipcc_wetlands_suplement', 'frel', 'year_frel',
       'fow'], dtype=object)

In [102]:
final_data_long.dtypes

location_id     int64
indicator      object
value          object
dtype: object

In [103]:
final_data_long.to_csv('../../../../data/international_status_data_UPDATED.csv', index=False)